In [2]:
import matplotlib
import numpy as np
import math
import itertools
import sklearn
import pydotplus
import warnings
import mysql.connector as myconnect
import pandas as pd
import sklearn.linear_model
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import tree
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from customLib1 import *
from sklearn.preprocessing import normalize
from sklearn import preprocessing
%matplotlib qt

In [3]:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
###################################### FUNCTIONS ######################################
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

### Query MySQL #######################################################################

def querySQL (database, query):
    config = {
    'user': 'root',
    'password': 'jim',
    'host': 'localhost',
    'database': database,
    'raise_on_warnings': True,
    }
    cnx = myconnect.connect(**config) 
    resultset = pd.read_sql_query(query,cnx)
    
    return resultset

#######################################################################################
### Give scores for subset of parameters ##############################################

def bestSubsetSelection(X_train, X_test, y_train, y_test, parCount, regType, alpha=0, poly=None):
    
    parameters = len(X[0])
    combinations = list(itertools.combinations(range(parameters),parCount))
    combinationsL = len(combinations)
    scores = np.empty([combinationsL])
    for i in range(0, combinationsL):
        
        reg = selectRegression(regType,alpha=alpha, poly=poly)
        reg.fit(X_train[:,combinations[i]], y_train)
        scores[i] = reg.score(X_test[:,combinations[i]], y_test)
        
    #Sort results (descending)
    indexMin = np.argsort(scores)
    sortedCombin =[combinations[u] for u in indexMin[::-1]]
    sortedScores =[scores[u] for u in indexMin[::-1]]
    #plt1 = plt.figure(1)
    #plt.plot(range(len(sortedScores)),sortedScores, 'ro')
    return sortedCombin, sortedScores

#######################################################################################
### Create full dummy dataframe #######################################################

def createFullDummyDF(positions, start35Box, start10Box):
    length = len(positions)
    a = np.empty([length],dtype=np.dtype(str,1))
    c = np.empty([length],dtype=np.dtype(str,1))
    t = np.empty([length],dtype=np.dtype(str,1))
    g = np.empty([length],dtype=np.dtype(str,1))
    dash = np.empty([length],dtype=np.dtype(str,1))
    a.fill('A')
    t.fill('T')
    c.fill('C')
    g.fill('G')
    dash.fill('A')
    dash[:start35Box[0]]='-'
    dash[(start35Box[0]+32):]='-'
    dataframe = pd.DataFrame(np.vstack((a,t,c,g,dash)),columns=positions)
    dummyTable = pd.get_dummies(dataframe)
    
    return dummyTable

######################################################################################
### Create position matrix ###########################################################

def createPositionMatrix(sequences):
    ntArray = np.empty([len(sequences),len(sequences[0])],dtype=np.dtype(str,1))
    sequences=np.array(sequences)
    for i in range(0,len(sequences)):
        ntArray[i] = [sequences[i][u] for u in range(0,len(sequences[i]))]
    
    return ntArray 

######################################################################################
### Create compatible position dataframe #############################################

def createPositionDF(seq, posRange, start35Box, start10Box):
    # Create Position Matrix
    posMatrix = createPositionMatrix(seq)
    # Convert to Dataframe
    posRangeCol = [str(x) for x in range(posRange[0],posRange[1])]
    dfPosMatrix = pd.DataFrame(posMatrix, columns = posRangeCol)

    # Create dummy Matrix
    dfDumPosMatrix =  pd.get_dummies(dfPosMatrix)


    return dfDumPosMatrix

#######################################################################################
### getParameter ######################################################################

def getParameters(parLabel, parRange):
    
    parameters = [np.zeros(parRange[u]) for u in range(len(parRange))]
 
 
    for i in range(len(parLabel)):
        if parLabel[i] is "alpha":
            parameters[i][:] = [math.pow(10,(u - np.around(parRange[i]/2))) for u in range(parRange[i])]
        elif parLabel[i] is "gamma":
            parameters[i][:] = [math.pow(10,(u - np.around(parRange[i]/2))) for u in range(parRange[i])]
        elif parLabel[i] is "C":
            parameters[i][:] = [math.pow(10,(u - np.around(parRange[i]/2))) for u in range(parRange[i])]
        elif parLabel[i] is "coef0":
            parameters[i][:] = [math.pow(10,(u - np.around(parRange[i]/2))) for u in range(parRange[i])]
        elif parLabel[i] is "epsilon":
            parameters[i][:] = [0+2/parRange[i]*u for u in range(parRange[i])]
        elif parLabel[i] is "max_depth":
            parameters[i][:] = [u+1 for u in range(parRange[i])]
        elif parLabel[i] is 'min_samples':
            parameters[i][:] = [u+1 for u in range(parRange[i])]
        elif parLabel[i] is 'max_features':
            parameters[i][:] = [int(u+2) for u in range(parRange[i])]
        else:
            return print("not a valid parameter")
    
    parEval = {parLabel[u]:parameters[u] for u in range(len(parLabel))}
    return parEval

########################################################################################
### Evaluate Multiple Parameters #######################################################

def evaluateMultiPar(X, y, modelPar, parLabel, parRange):
    
    #data prep
    parEval = getParameters(parLabel,parRange)
    yMin, yMax = np.min(parEval[parLabel[0]]), np.max(parEval[parLabel[0]])
    xMin, xMax = np.min(parEval[parLabel[1]]), np.max(parEval[parLabel[1]])
    
    #model fitting
    reg = selectRegression(**modelPar)
    GS = GridSearchCV(reg, parEval)
    GS.fit(X,y)

    #plotting
    plt.figure("Model accuracy ifv model parameters")
    plt.imshow(GS.cv_results_["mean_test_score"].reshape([parRange[0],parRange[1]]), cmap='hot', interpolation='none', 
              vmin=0, vmax=np.max(GS.cv_results_["mean_test_score"]),  extent=[xMin,xMax,yMin,yMax], aspect="auto", 
              )
    plt.colorbar()
    plt.xlabel(parLabel[1]), plt.xscale('log')
    plt.ylabel(parLabel[0]), plt.yscale('log')

########################################################################################
### Evaluate Single Parameter #################################################################

def evaluateSinglePar(X_train, X_test, y_train, y_test, parModel, parLabel, parRange):

    parEval = getParameters([parLabel], parRange)
    scores = np.zeros(parRange)

    for i in range(len(parEval[parLabel])):
        parEvalIt = {parLabel:parEval[parLabel][i]}
        reg = selectRegression(**parModel,**parEvalIt)
        reg.fit(X_train,y_train)
        scores[i] = reg.score(X_test,y_test)
    
    optimalPar = parEval[parLabel][np.argmax(scores)]
    
    return scores, optimalPar

##########################################################################################
### Evaluate Parameter ################################################################

def evaluateScore(X_train, X_test, y_train, y_test, parModel):

    reg = selectRegression(**parModel)
    reg.fit(X_train,y_train)
    score = reg.score(X_test,y_test)
    y_pred = reg.predict(X_test)
 
    return score, y_pred

##########################################################################################
### Extract ATCG fractions region ########################################################

def extractATCGpercentage(box):
    boxLength = len(box[1])
    
    fractionA= [u.count('A')/boxLength for u in box]
    fractionT= [u.count('T')/boxLength for u in box]
    fractionC= [u.count('C')/boxLength for u in box]
    fractionG= [u.count('G')/boxLength for u in box]
    
    return fractionA, fractionT, fractionC, fractionG

##########################################################################################
### K-fold regression cross validation ###################################################

def KfoldCV(X,y,k,parModel, parLabel ,parRange):
    
    y = np.array(y)
    kf = KFold(n_splits=k)
    scoresPar = np.zeros([k,parRange])
    OutI=np.bool([0])
    OutV=np.empty([0])
    optimalPar = np.zeros([k])
    index=0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        scoresPar[index,:], optimalPar[index] = evaluateSinglePar(X_train, X_test, y_train, y_test, parModel, parLabel, [parRange])
        score, y_pred = evaluateScore(X_train, X_test, y_train, y_test, {**parModel, parLabel: optimalPar[index]})
        fig = plt.figure("pred/true: K-fold")
        plt.plot(y_test,y_pred, 'bo')
        plt.xlabel("True label")
        plt.ylabel("Predicted label")
        index+=1
    
    #scores = np.transpose(scoresM)
    plt.plot([0,6],[0,6], ls="--", c=".3")
    plt.figure("Cross Validation")
    plt.imshow(scoresPar, cmap='hot', vmin=0 ,interpolation='nearest')
    plt.colorbar()
    plt.figure("Cross Validation: Normalized")
    plt.imshow(sklearn.preprocessing.normalize(scoresPar, axis=1),cmap='hot', vmin=0,interpolation='nearest')
    plt.colorbar()

    return scoresPar, optimalPar

##########################################################################################
### Leave one out accuracy estimation ####################################################

def leaveOneOut(X,y,regType,alpha=1):
    y=np.array(y)
    X=np.array(X)
    loo = LeaveOneOut()
    scores = np.zeros([len(X)])
    index = 0
    for train_index,test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train,y_train, X_test, y_test)
        reg = selectRegression(regType, alpha=alpha)
        reg.fit(X_train,y_train)
        scores[index] = reg.score(X_test,y_test)
        #print(reg.score(X_test,y_test))
        index += 1
    
        return scores

########################################################################################
### Merge multiple regions in one position dataframe####################################

def mergePositionDF(df1,df2):
    
    df3 = pd.concat([df1, df2], axis=1)
    if len(df3)>len(df2):
        print("WARNING: OVERLAPPING POSITIONS")
    else:
        return df3
    

########################################################################################
### K-fold nested cross validation #####################################################

def nestedKfoldCV(X,y,k,kInner, parModel, parLabel,parRange):
    y = np.array(y)
    kf = KFold(n_splits=k)
    kfInner = KFold(n_splits=kInner)
    scoresM = np.zeros([k,kInner,parRange])
    scoresCV = np.zeros([k]) 
    optimalParM = np.zeros([k,kInner])
    index=0

    for train_index, test_index in kf.split(X):     
        indexI=0
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        splitLib = list(kfInner.split(X_train))
        for train_indexI, test_indexI in splitLib:
            X_trainI, X_testI = X_train[train_indexI], X_train[test_indexI]
            y_trainI, y_testI = y_train[train_indexI], y_train[test_indexI]
            scoresM[index,indexI,:], optimalParM[index,indexI] = evaluateSinglePar(X_train, X_test, y_train, y_test, parModel, parLabel, [parRange])
            indexI += 1
        
        scoresCV[index], y_pred = evaluateScore(X_train, X_test, y_train, y_test, {**parModel, parLabel:  stats.mode(optimalParM[index])[0][0]})
        fig = plt.figure("pred/true: Nested K-fold")
        plt.plot(y_test,y_pred, 'bo')
        plt.plot([0,6],[0,6],ls="--", c=".3")

        index+=1
        
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    plt.show()
  

    return scoresCV, optimalParM, scoresM

#########################################################################################
### Plot Histograms ######################################################################

def plotHistograms(yData,blocks):
    #yData = np.array(yData)
    yDataSqrt = [math.sqrt(y) for y in yData]
    yDataLog = [math.log(10,y) for y in yData]
    yDataQuad = np.power(yData,2)
    plot = plt.figure("Histograms")

    plt.subplot(221)
    plt.hist(yData,blocks,normed=1)
    plt.title('Histogram of y values')
    plt.subplot(222)
    plt.hist(yDataSqrt,blocks,normed=1)
    plt.title('Histogram of squared y values')
    plt.subplot(223)
    plt.hist(yDataLog,blocks,normed=1)
    plt.title('Histogram of log y values')
    ax4 = plt.subplot(224)
    plt.hist(yDataQuad,blocks,normed=1)
    plt.title('Histogram of quadratic y values')\
    
    wm = plt.get_current_fig_manager()

##########################################################################################
### Plot Position coefficients ###########################################################

def plotPositionCoefficients(linReg, dfDumPosMatrix, positions, results=''):
    posRangeCol = posRangeCol = [str(x) for x in range(positions[0],positions[1])]
    
    normCoef = linReg.coef_/linReg.coef_.max()
    normInt = linReg.intercept_/linReg.coef_.max()
    #normCoef = linReg.coef_
    #normInt = linReg.intercept_
    
    #print(normCoef, dfDumPosMatrix.columns)
    dfResultBox = pd.DataFrame([normCoef],columns=dfDumPosMatrix.columns.values)
    
    dfTemplateDummyBox = createFullDummyDF(posRangeCol)
    dfFinalBox = pd.DataFrame(np.nan, index=[0],columns=dfTemplateDummyBox.columns)

    colNamesResult = dfResultBox.columns

    for i in colNamesResult:
        if i not in  ['spacer','upSeq']:
            dfFinalBox[i] = dfResultBox[i]

    # Extract values per nt
            
    normA = separateNtData('A',dfFinalBox)
    normC = separateNtData('C',dfFinalBox)
    normT = separateNtData('T',dfFinalBox)
    normG = separateNtData('G',dfFinalBox)
    
    
    plt.figure("Position Matrix")
    bar_width = 0.15
    index = np.arange(positions[0],positions[1])
    
    rectsU = plt.bar(positions[0]-0.6, normCoef[-1], bar_width, color='orange', label='Up-Seq')
    rectsS = plt.bar(positions[0]-0.3, normCoef[-2], bar_width, color='purple', label='Spacer')
    rects0 = plt.bar(positions[0]-1, normInt, bar_width, color='black', label="Intercept")
    rects1 = plt.bar(index, normA, bar_width, label='A')
    rects2 = plt.bar(index + bar_width, normG, bar_width, color='r', label='G')
    rects3 = plt.bar(index + 2*bar_width, normC, bar_width, color='g', label='C')
    rects4 = plt.bar(index + 3*bar_width, normT, bar_width, color='y', label='T')
    #plt2 = plt.figure(2)
    plt.xlabel('Position')
    plt.ylabel('Coefficients')
    plt.title('Coefficients of every nucleotide at given position: ')
    plt.axis([positions[0]-1, positions[1]+1, -2, 2])
    plt.legend(prop={'size':15})

##########################################################################################
### Plot Predicted Scores to real scores #################################################

def plotPredictedToReal(yReal,yPred):
    plt.figure(ScatterPvsR)
    plt.plot(yReal,yPred,'ro')
     
        
##########################################################################################
### Region selection #####################################################################

def regionSelect(dataset, seqRegions, posRange, test=None):
    
    
    

    # Selecting regions
    sequences= dataset[:,0]    
    start35Box = dataset[:,2]
    start10Box = dataset[:,3]
    
    posRange = [-42,1]
    difLength = -35-posRange[0]-start35Box
    sequenceAligned = ["-" *difLength[u]+sequences[u] if difLength[u]>=0 else sequences[u][abs(difLength[u]):] for u in range(np.shape(sequences)[0]) ]
    start35Box = np.array([start35Box[u]+difLength[u] for u in range(np.shape(sequences)[0])])
    start10Box = np.array([start10Box[u]+difLength[u] for u in range(np.shape(sequences)[0])])

    maxLength = max([len(sequenceAligned[u]) for u in range(np.shape(sequenceAligned)[0])])
    difLength = [maxLength - len(sequenceAligned[u]) for u in range(np.shape(sequenceAligned)[0])]
    sequences = [sequenceAligned[u]+ '-'*difLength[u] for u in range(np.shape(sequenceAligned)[0]) ]
    
    
    reg35 = np.array(seqRegions[0])
    reg10 = np.array(seqRegions[1])
    
    
    box35 = selectRegions(sequences, reg35, start35Box)
    box10 = selectRegions(sequences, reg10, start10Box)
    spacer = start10Box-start35Box-6
    
    spacerM = [u-17 if u-17>0 else 0 for u in spacer]
    spacerL = [abs(u-17) if u-17<0 else 0 for u in spacer]
    
    spacerBox = pd.DataFrame({'spacer_more':spacerM,'spacer_less':spacerL})
   
    # Creating features

    positionBox35 = createPositionDF(box35, reg35-35, start35Box, start10Box)
    positionBox10 = createPositionDF(box10, reg10-12, start35Box, start10Box)

    positionBoxSS = mergePositionDF(positionBox35,positionBox10)
      
    posRangeCol = [str(x) for x in range(posRange[0],posRange[1])]
    dfTemplateDummyBox = createFullDummyDF(posRangeCol, start35Box, start10Box)

    dfFinalBox = pd.DataFrame(0, range(len(dataset)),columns=dfTemplateDummyBox.columns)
    
    colNamesResult = positionBoxSS.columns
    for i in colNamesResult:
        if i in dfFinalBox.columns:
            dfFinalBox[i] = positionBoxSS[i]
    
    positionBox = mergePositionDF(dfFinalBox, spacerBox)

        
    return positionBox, spacer
    

##########################################################################################
### Regression selection #################################################################

def selectRegression(regType, poly=None, kernel=None, alpha=0.0001, gamma=0.1, epsilon=1.95, coef0=1,  
                     fitInt=True, norm=True, max_depth=None, max_features=None, min_samples_split = 2, 
                     treeCount = 100, C=1):
    if kernel:
        if regType is "ridge":
            reg = KernelRidge(alpha=alpha, gamma=gamma, kernel=kernel, coef0=coef0)
        if regType is "SVC":
            reg = sklearn.svm.SVC(C=C, kernel=kernel, gamma=gamma, coef0=coef0, epsilon=epsilon, degree=poly)
        if regType is "SVR":
            reg = sklearn.svm.SVR(C=C, kernel=kernel, gamma=gamma, coef0=coef0, epsilon=epsilon, degree=poly)
                    
    elif poly:
        if regType is "OLS":
            reg = make_pipeline(PolynomialFeatures(poly), LinearRegression(fit_intercept=fitInt, normalize=norm))
        if regType is "ridge":
            reg = make_pipeline(PolynomialFeatures(poly), sklearn.linear_model.Ridge(alpha= alpha, normalize=True))
        if regType is "lasso":
            reg = make_pipeline(PolynomialFeatures(poly), sklearn.linear_model.Lasso(alpha= alpha, normalize=True))
        if regType is "huber":
            reg = make_pipeline(PolynomialFeatures(poly), sklearn.linear_model.HuberRegressor(fit_intercept=fitInt, epsilon=epsilon, alpha=alpha))

    else: 
        if regType is "OLS":
            reg = LinearRegression(fit_intercept=fitInt, normalize=norm)
        if regType is "ridge":
            reg = sklearn.linear_model.Ridge(alpha= alpha, normalize=True)
        if regType is "lasso":
            reg = sklearn.linear_model.Lasso(alpha= alpha, normalize=True)
        if regType is "huber":
            reg = sklearn.linear_model.HuberRegressor(fit_intercept=fitInt, alpha=alpha, epsilon=epsilon)
        if regType is "treeReg":
            reg = tree.DecisionTreeRegressor(max_depth= max_depth, max_features=max_features, min_samples_split = min_samples_split)
        if regType is "treeClass":
            reg = tree.DecisionTreeClassifier(max_depth = max_depth, max_features=max_features, min_samples_split = min_samples_split)
        if regType is "forestReg":
            reg = RandomForestRegressor(n_estimators = treeCount, max_depth = max_depth, max_features= max_features, min_samples_split = min_samples_split)
        if regType is "forestClass":
            reg = RandomForestClassifier(n_estimators = treeCount, max_depth = max_depth, max_features= max_features, min_samples_split = min_samples_split)
    
    return reg

##########################################################################################
### Select regions of interest ###########################################################

def selectRegions(sequences, positions, referencePoint=None):
    if referencePoint.all == None:
        selectedRegion = [sequences[u][positions[0]:positions[1]] for u in range(0,len(sequences))]
    else:
        selectedRegion = [sequences[u][positions[0]+referencePoint[u]:positions[1]+referencePoint[u]] for u in range(0,len(sequences))]
    
    return selectedRegion

##########################################################################################
### Extract nucleotide values from resultset #############################################

def separateNtData(colSubStr, Data):
    Index = [c for c in Data.columns if colSubStr in c]
    subData = np.squeeze(Data[Index].values)
    
    return subData

##########################################################################################

In [16]:
dash = np.empty([35],dtype=np.dtype(str,1))
dash.fill('a')
dash[:3]='b'


dash

array(['b', 'b', 'b', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'], 
      dtype='<U1')

In [7]:
#   spacer length
#   frequency AGCT
#   position base
#   %AT / %GC
#   difference 

#   binding affinity

#dataset vergroten adhv bootstrap
#Histogram 
#Principal component analysis -> kleurenmap adhv expressiematrix -> kleurenmap hoog contrast
#normalize ridge/lasso
#support vector machines -> kernel voor sequenties.

"""
Best Feature Selection Method?
Alpha voor CV
Optimal alpha
"""

### Check voor identieke sequenties
### Optimalisatie
### Spearman Rank Coefficient
### Kendel Tau
### Covariantie

#variantie binnenin dataset
# gaussian processer
# Promoter data Composability of regulatory sequences controlling transcription and translation in Escherichia coli

# 4de machts wortel, kwadraat, dat delen door resultaat


'\nBest Feature Selection Method?\nAlpha voor CV\nOptimal alpha\n'

In [39]:
# Query sequences and fluorescence level
plt.close('all')

## RAND + MODULATED
dfDatasetOrder = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
## RAND
#dfDatasetOrder = querySQL('mutalik_prom_lib','SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd  FROM rand_prom_lib')
## MODULATED
#dfDataset = querySQL('mutalik_prom_lib','SELECT sequence, mean_fluo, 35boxstart, 10boxstart FROM mod_prom_lib')

dfDataset = dfDatasetOrder.reindex(np.random.permutation(dfDatasetOrder.index))

dataset = dfDataset.values
#print(dfDataset)

sequences= dataset[:,0]
yData = dataset[:,1]
start35Box = dataset[:,2]
start10Box = dataset[:,3]
ID = dataset[:,4]
mean_fluo_sd = dataset[:,5]
#


# Selecting regions


positionBox, spacer = regionSelect(dataset, [[0,12],[-8,12]], [-35,1])

positionMatrix = positionBox.values
# Plot linear regression

#plotHistograms(yData,15)
np.set_printoptions(threshold=np.nan)

In [41]:
plt.close('all')

#----parEvalS --------------
Xdf = positionBox
#X = preprocessing.scale(positionMatrix, axis=1)
X = positionMatrix
y = [math.sqrt(math.sqrt(u)) for u in yData]

regType = 'ridge'
poly= 3

kernel = 'poly'
parLabel = ['alpha','coef0']
parRange = [10,10] 

testSize = 0.20 
k = 5 
kInner = 5
pdf = None


##OPTIONAL

treeCount = 100
gamma = 0.1
alpha = 0.1
coef0 = 0.00
#---------------------------
penalty = 0.1
epsilon = 1.95

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "treeCount":treeCount, "gamma":gamma, "coef0":coef0}


#

if len(parLabel) is 1: 
    scoresParCV, optimalParCV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")

if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


    

In [42]:
## Show Out

XnoOut = positionMatrix[np.invert(OutI),:]
YnoOut = np.extract(np.invert(OutI),y)

IDout, seqOut, scoreOut = np.extract(OutI,ID),np.extract(OutI,sequences),np.extract(OutI, OutV)
fluoOut, fluoSdOut, spacerOut = np.extract(OutI,yData), np.extract(OutI, mean_fluo_sd),np.extract(OutI,spacer)
difLength = np.max(np.extract(OutI,start10Box))-np.extract(OutI,start10Box)
sequenceAligned = ["-" *difLength[u]+seqOut[u] for u in range(np.shape(seqOut)[0])]
np.transpose(np.vstack((IDout, sequenceAligned, scoreOut, fluoOut, fluoSdOut, spacerOut)))

#[[sequences[u], OutV[u]] for u, i in enumerate(OutI) if i==False]

NameError: name 'OutI' is not defined

In [43]:
plt.close('all')

#----parEvalS --------------
Xdf = positionBox
#X = XnoOut
#y = YnoOut

regType = 'ridge'
poly= 3

kernel = 'poly'
parLabel = ['alpha']
parRange = [15] 

testSize = 0.20 
k = 5 
kInner = 5
pdf = None


##OPTIONAL

treeCount = 100
gamma = 0.1
alpha = 0.1
coef0 = 0.01
#---------------------------
penalty = 0.1
epsilon = 1.95

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "treeCount":treeCount, "gamma":gamma, "coef0":coef0}

test = sklearn.svm.SVR( C=penalty, kernel=kernel, gamma=gamma, coef0=coef0, epsilon=epsilon, degree=poly)
test.fit(X,y)
#

if len(parLabel) is 1: 
    scoresParCV, optimalParCV, OutI, OutV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")

if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


ValueError: not enough values to unpack (expected 4, got 2)

In [45]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM anderson_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-35,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[0,12],[-12,12]], posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[0,12],[-12,12]], posRange, positionBoxTrain)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

print(np.size(X))
print(np.size(Xtest))


#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel }
"""
parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------
#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.001, "alpha": 0.001, "coef0":0.001 } 
reg = selectRegression(**parModelTweak)
#reg = GS.best_estimator_
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')



46990
3515
[['TTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC' 1.0 0.959285906025329]
 ['TTGACAGCTAGCTCAGTCCTAGGTACTGTGCTAGC' 0.86 0.9223489602844605]
 ['TTGACAGCTAGCTCAGTCCTAGGTATTGTGCTAGC' 0.72 1.0419421285064305]
 ['TTTACAGCTAGCTCAGTCCTAGGTATTATGCTAGC' 0.7 0.9860207058854685]
 ['TTGACGGCTAGCTCAGTCCTAGGTATAGTGCTAGC' 0.58 1.0864933433798911]
 ['TTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGC' 0.56 0.9406858386374011]
 ['CTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGC' 0.51 1.136621370809569]
 ['TTTACGGCTAGCTCAGTCCTAGGTATAGTGCTAGC' 0.47 0.9048435579522482]
 ['TTTACGGCTAGCTCAGCCCTAGGTATTATGCTAGC' 0.36 0.9167794384497946]
 ['TTTACGGCTAGCTCAGTCCTAGGTACAATGCTAGC' 0.33 0.9057306125453374]
 ['TTTACGGCTAGCTCAGTCCTAGGTACTATGCTAGC' 0.24 0.7763212956297119]
 ['TTGACAGCTAGCTCAGTCCTAGGGACTATGCTAGC' 0.16 0.9181584583331551]
 ['TTTATAGCTAGCTCAGCCCTTGGTACAATGCTAGC' 0.15 0.899667490467349]
 ['TTTATGGCTAGCTCAGTCCTAGGTACAATGCTAGC' 0.1 0.8098252368958583]
 ['TTGACAGCTAGCTCAGTCCTAGGGATTGTGCTAGC' 0.06 0.916751556510665]
 ['TTTACAGCTAGCTCAGTC

In [26]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM brewster_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-40,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-5,13],[-6,12]],posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-5,13],[-6,12]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]


#print(np.array(positionBoxTrain.columns))
#print(np.array(positionBoxTest.columns))

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3


#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel }

"""parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.0001, "alpha":0.1, "coef0":0.1 } 
####
reg = selectRegression(**parModelTweak)
#reg = GS.best_estimator_
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')



[['CAGGCTTTACACTTTATGCTTCCGGCTCGGTTGTAGTGTGG' 81.46 2.252786289810921]
 ['CAGGCCTCAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 76.0 2.2579654675406733]
 ['CAGGCCTTAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 69.93 2.258039776750028]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGTTAAAGTGTGG' 69.81 2.262938208700772]
 ['CAGGCTTAAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 69.03 2.264208960552707]
 ['TCGAGCTCAGACTTTATGCTTCCGGCTCGTATAATGTGTGG' 64.35 2.268143731598088]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGATAACGTGTGG' 62.58 2.2686194838553897]
 ['CAGGCATTACACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 57.83 2.263616397531156]
 ['TCGAGTTAAGACTTTATGCTTCCGGCTCGTATAATGTGTGG' 57.38 2.274405600470362]
 ['CAGGCTTTACACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 53.45 2.270314841106836]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGATAATGTGTGG' 51.76 2.2754154184724538]
 ['TCGTGTTTACCCTTTATGCTTCCGGCTCGTATAATGTGTGG' 49.57 2.276573606087575]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGAATAATGTGTGG' 46.74 2.2760239028840727]
 ['TCGAGATTACACTTTATGCTTCCGGCTCGTATAATGTGTGG' 46.18 2.2738050293983383]
 [

In [4]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM inbio_prom_lib WHERE length(sequence)>115')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-45,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-7,12],[-8,11]],posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-7,12],[-8,11]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel }

"""parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.0001, "alpha": 10000, "coef0":0.1 } 
####
#reg= GS.best_estimator_
reg = selectRegression(**parModelTweak)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
#print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')

SpearmanrResult(correlation=0.12627265162638146, pvalue=0.37238869055788715)


In [13]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM hammer_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-40,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-7,14],[-9,12]], posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-7,14],[-9,12]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

"""parModel = {"regType":regType, "poly":poly, "kernel":kernel }

parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)
"""
#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":10, "alpha":0.1, "coef0":0.1 } 
####
reg = selectRegression(**parModel)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')





[['CTTTGGCAGTTTATTCTTGACATGTAGTGAGGGGGCTGGTATAATCACATAGTACTGTT' 528.0
  3.036887037555399]
 ['CATGTGGGAGTTTATTCTTGACACAGATATTTCCGGATGATATAATAACTGAGTACTGTT' 280.0
  2.716241258271957]
 ['TATGCGGTAGTTTATTCTTGACATGACGAGACAGGTGTGGTATAATGGGTCTAGATTAGG' 134.0
  3.0525260233182774]
 ['CATATACAAGTTTATTCTTGACACTAGTCGGCCAAAATGATATAATACCTGAGTACTGTT' 101.0
  2.1881650165053452]
 ['CATAGAGAAGTTTATTCTTGACAGCTAACTTGGCCTTTGATATAATACATGAGTACTGTT' 92.0
  2.3769709971684048]
 ['CATACGGGAGTTTATTCTTGACATATTGCCGGTGTGTTGGTATAATAACTTAGTACTGTT' 60.0
  3.6688988383684578]
 ['CATAGTCTAGTTTATTCTTGACACGCGGTCCATTGGCTGGTATAATAATTTAGTACTGTT' 56.0
  3.089457607879062]
 ['CATGACAGAGTTTATTCTTGACAGTATTGGGTTACTTTGGTATAATAGTTGAGTACTGTT' 42.0
  2.581752498639796]
 ['CATCGGGTAGTTTATTCTTGACAATTAAGTAGAGCCTGATATAATAGTTCAGTACTGTT' 34.0
  2.5285787706578793]
 ['CATGATGTAGTTTATTCTTGACACTGAGAGGGCCTCTTGATATAATAGTTGAGTACTGTT' 33.0
  2.5191920880142575]
 ['CATGGGTGAGTTTATTCTTGACAGTGCGGCCGGGGGCTGATATCATAGCAGAGTACTATT' 22.0
  2.32087564

In [32]:
plt.close('all')

#----PARAMETERS -----
Xdf = positionBox
X = XnoOut
y = YnoOut


regType = 'ridge'
poly= 3
gamma = 1
coef0 = 1

kernel = 'poly'
parLabel = ['alpha']
parRange = [15] 

testSize = 0.20 
k = 5 
kInner = 5
pdf = None
#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "gamma":gamma, "coef0":coef0}


if len(parLabel) is 1: 
    scoresParCV, optimalParCV, OutI, OutV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")
if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


K FOLD CV 
---------- 

 Maximum Score:  0.8636674365 
 Mean optimal score:  0.82419440891 
 sd optimal scores:  0.056555113388595396 
 Optimal parEval:
 [  1000.  10000.   1000.   1000.  10000.] 
 parEval Scores:
 [[ 0.70339157  0.70339157  0.70339157  0.70339157  0.70339163  0.70339216  0.70339752  0.70345111  0.70398418
   0.70905269  0.74225203  0.78447419  0.71069898  0.42680514 -0.86323757]
 [ 0.77386117  0.77386117  0.77386117  0.77386117  0.7738612   0.77386151  0.77386461  0.77389557  0.77420363
   0.77713337  0.7966842   0.83691832  0.86366744  0.6795331  -0.44599599]
 [ 0.81285496  0.81285496  0.81285496  0.81285496  0.81285499  0.81285519  0.81285725  0.81287783  0.8130817   0.81494989
   0.8248565   0.83073847  0.79828143  0.54095783 -0.91545982]
 [ 0.81226191  0.81226191  0.81226191  0.81226192  0.81226193  0.81226204  0.81226322  0.81227492  0.81239128
   0.81348609  0.82016728  0.82095822  0.75534359  0.52605366 -0.71299687]
 [ 0.7616594   0.7616594   0.7616594   0.7616

In [11]:

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM hammer_prom_lib')

dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
dataset = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


sequences= dataset[:,0]
yData = dataset[:,1]
start35Box = dataset[:,2]
start10Box = dataset[:,3]

ID = dataset[:,4]

posRange = [-35,1]
XtestBox, spacerTest = regionSelect(dataset, [[0,10],[-6,12]], test=positionBox)
Xtest = XtestBox.values

regType = 'ridge'
poly= 3
gamma = 1
coef0 = 1
alpha = 1000
kernel = 'poly'

testSize = 0.20 
k = 5 
kInner = 5
pdf = None
#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "gamma":gamma, "coef0":coef0, "alpha":alpha}

reg = selectRegression(**parModel)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)

print(np.transpose(np.vstack((sequences,yData,rankPredict))))

print(stats.spearmanr(yData,rankPredict))
print(stats.spearmanr(rankPredict,yData))



[['CTTTGGCAGTTTATTCTTGACATGTAGTGAGGGGGCTGGTATAATCACATAGTACTGTT' 528.0 5.708700659453884]
 ['CATGTGGGAGTTTATTCTTGACACAGATATTTCCGGATGATATAATAACTGAGTACTGTT' 280.0 5.708700659453884]
 ['TATGCGGTAGTTTATTCTTGACATGACGAGACAGGTGTGGTATAATGGGTCTAGATTAGG' 134.0 5.708700659453884]
 ['CATATACAAGTTTATTCTTGACACTAGTCGGCCAAAATGATATAATACCTGAGTACTGTT' 101.0 5.708700659453884]
 ['CATAGAGAAGTTTATTCTTGACAGCTAACTTGGCCTTTGATATAATACATGAGTACTGTT' 92.0 5.708700659453884]
 ['CATACGGGAGTTTATTCTTGACATATTGCCGGTGTGTTGGTATAATAACTTAGTACTGTT' 60.0 5.708700659453884]
 ['CATAGTCTAGTTTATTCTTGACACGCGGTCCATTGGCTGGTATAATAATTTAGTACTGTT' 56.0 5.708700659453884]
 ['CATGACAGAGTTTATTCTTGACAGTATTGGGTTACTTTGGTATAATAGTTGAGTACTGTT' 42.0 5.708700659453884]
 ['CATCGGGTAGTTTATTCTTGACAATTAAGTAGAGCCTGATATAATAGTTCAGTACTGTT' 34.0 5.399529980199178]
 ['CATGATGTAGTTTATTCTTGACACTGAGAGGGCCTCTTGATATAATAGTTGAGTACTGTT' 33.0 5.708700659453886]
 ['CATGGGTGAGTTTATTCTTGACAGTGCGGCCGGGGGCTGATATCATAGCAGAGTACTATT' 22.0 4.632584202374565]
 ['CATAGAACAGTTTATT

In [121]:
""" LINEAR REGRESSION USING OTHER ELEMENTS

# Query sequences, fluorescence level, reference regions

resultset =  np.squeeze(querySQL('mutalik_prom_lib','SELECT sequence, mean_fluo, 35boxstart, 10boxstart FROM rand_prom_lib').values)

sequences= resultset[:,0]
score = resultset[:,1]
start35Box = resultset[:,2]
start10Box = resultset[:,3]

# Selecting regions

box10 = selectRegions(sequences, [0,6], start10Box)
box35 = selectRegions(sequences, [0,6], start35Box)
spacer = start10Box-start35Box-6

# Creating features


fractionA_10, fractionT_10, fractionC_10, fractionG_10 = extractATCGpercentage(box10)
fractionA_35, fractionT_35, fractionC_35, fractionG_35 = extractATCGpercentage(box35)

fractionGC = [(o+p+q+r)/4 for o,p,q,r in zip(fractionC_10,fractionG_10,fractionC_35,fractionG_35)]
fractionAT = [(o+p+q+r)/4 for o,p,q,r in zip(fractionA_10,fractionT_10,fractionA_35,fractionT_35)]


featureMatrix = np.transpose(np.array([fractionGC,spacer]))


#print(featureMatrix)

print(np.shape(spacer),np.shape(score))
# Fit linear regression

linReg = LinearRegression(fit_intercept=True, normalize=True)
linReg.fit(featureMatrix, score)

#linReg.fit(np.squeeze(spacer), np.squeeze(score))

print(linReg.coef_)
print(linReg.intercept_)


# Plot coefficients
index = np.arange(np.shape(featureMatrix)[1])
coefPlot = plt.bar(index, linReg.coef_, 0.10 )
intPlot = plt.bar(-1, linReg.intercept_, 0.10)
plt.axhline(0, color='black')
plt.axvline(0, color='black')


(138,) (138,)
[-311.19084743   68.32928228]
-1000.49146613


In [ ]:
######################### EXPERIMENTAL GROUNDS ###############################################


"""
topScores= np.empty([8,8])
topCombinations = np.empty([8,8])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)
logIt = [math.pow(10,u) for u in range(-3,5)]
for i in logIt:
    index = int(math.log10(i)+3)
    combinations, results = bestSubsetSelection(X_train, X_test, y_train, y_test,2, 'ridge', alpha=i)
    print(results[0:8])
    print(combinations[0:8])"""








